In [0]:
import warnings
# Squash warning messages
#warnings.showwarning = lambda *args, **kwargs: None

import pandas as pd
import sklearn
from sklearn import cluster
# from sklearn import ensemble
# from sklearn import metrics
# from sklearn import model_selection
# from sklearn.metrics import davies_bouldin_score

import numpy as np
import math
import time

from scipy import stats

# Import the necessary functions for reading iww sql db
from pyspark.sql.functions import *

# Use arrow conversion for spark DF to pandas DF and vice versa
spark.conf.set("spark.sql.execution.arrow.enabled", "true")

print("Pandas ", pd.__version__)
print("Numpy ", np.__version__)

#Importing Pandas
#Raw_Total_Data = pd.read_cs//v(r"C:\Users\ryan.hagan\Desktop\01_Preprocessed_WaterQuality_Data.csv", encoding = "ISO-8859-1")
                             
#Raw_Target_Data = pd.read_csv(r"C:\Users\ryan.hagan\Desktop\01_Preprocessed_WaterQuality_Data.csv", encoding = "ISO-8859-1")

Pandas 0.19.2
Numpy 1.11.1

In [0]:

#Reading new IWW Water Quality data table
iww_quality_SDF = spark.read.format("jdbc") \
	.option("driver", "com.microsoft.sqlserver.jdbc.SQLServerDriver")\
	.option("url", "jdbc:sqlserver://sqlqutarmsvruw2003.database.windows.net;databaseName=edsqutarmsqluw2005") \
	.option("dbtable", "RAW_SAMPLE_DATA") \
	.option("user", "ABO_IWWD_Admin") \
	.option("password","!WWD_ad_2019").load()

iww_toxicity_SDF = spark.read.format("jdbc") \
	.option("driver", "com.microsoft.sqlserver.jdbc.SQLServerDriver")\
	.option("url", "jdbc:sqlserver://sqlqutarmsvruw2003.database.windows.net;databaseName=edsqutarmsqluw2005") \
	.option("dbtable", "RAW_TOXICITY_DATA") \
	.option("user", "ABO_IWWD_Admin") \
	.option("password","!WWD_ad_2019").load()


#Using Pandas dataframes with arrow conversion
Raw_Total_Data = iww_quality_SDF.select("*").toPandas()
Raw_Target_Data = Raw_Total_Data
Raw_Toxicity_Data = iww_toxicity_SDF.select("*").toPandas()


Raw_Toxicity_Data.drop('Mapsite', axis=1, inplace=True)
Raw_Toxicity_Data.drop('Outfall_Status', axis=1, inplace=True)

#Raw_Toxicity_Data.head()


Joined_water_data=pd.merge(left=Raw_Total_Data, right=Raw_Toxicity_Data, left_on='Sample_Date', right_on='Sample_Date', how='outer' )



# # Write the Bridge dataframe to a new table in the database
joined_SDF = spark.createDataFrame(Joined_water_data)
joined_SDF.write.format("jdbc").mode("overwrite") \
    .option("driver", "com.microsoft.sqlserver.jdbc.SQLServerDriver")\
    .option("url", "jdbc:sqlserver://sqlqutarmsvruw2003.database.windows.net;databaseName=edsqutarmsqluw2005") \
    .option("dbtable", "RAW_QUALITY_DATA") \
    .option("user", "ABO_IWWD_Admin") \
    .option("password", "!WWD_ad_2019").save()

--------------------------------------------------------------------------- 
 Py4JJavaError Traceback (most recent call last)
 <command-3644325364902747> in <module> () 
 1 
 2 #Reading new IWW Water Quality data table 
 ----> 3 iww_quality_SDF = spark . read . format ( "jdbc" ) . option ( "driver" , "com.microsoft.sqlserver.jdbc.SQLServerDriver" ) . option ( "url" , "jdbc:sqlserver://sqlqutarmsvruw2003.database.windows.net;databaseName=edsqutarmsqluw2005" ) . option ( "dbtable" , "RAW_SAMPLE_DATA" ) . option ( "user" , "ABO_IWWD_Admin" ) . option ( "password" , "!WWD_ad_2019" ) . load ( ) 
 4 
 5 iww_toxicity_SDF = spark . read . format ( "jdbc" ) . option ( "driver" , "com.microsoft.sqlserver.jdbc.SQLServerDriver" ) . option ( "url" , "jdbc:sqlserver://sqlqutarmsvruw2003.database.windows.net;databaseName=edsqutarmsqluw2005" ) . option ( "dbtable" , "RAW_TOXICITY_DATA" ) . option ( "user" , "ABO_IWWD_Admin" ) . option ( "password" , "!WWD_ad_2019" ) . load ( ) 

 /databricks/spark/python/pyspark/sql/readwriter.py in load (self, path, format, schema, **options) 
 170 return self . _df ( self . _jreader . load ( self . _spark . _sc . _jvm . PythonUtils . toSeq ( path ) ) ) 
 171 else : 
 --> 172 return self . _df ( self . _jreader . load ( ) ) 
 173 
 174 @ since ( 1.4 ) 

 /databricks/spark/python/lib/py4j-0.10.7-src.zip/py4j/java_gateway.py in __call__ (self, *args) 
 1255 answer = self . gateway_client . send_command ( command ) 
 1256 return_value = get_return_value(
 -> 1257 answer, self.gateway_client, self.target_id, self.name)
 1258 
 1259 for temp_arg in temp_args : 

 /databricks/spark/python/pyspark/sql/utils.py in deco (*a, **kw) 
 61 def deco ( * a , ** kw ) : 
 62 try : 
 ---> 63 return f ( * a , ** kw ) 
 64 except py4j . protocol . Py4JJavaError as e : 
 65 s = e . java_exception . toString ( ) 

 /databricks/spark/python/lib/py4j-0.10.7-src.zip/py4j/protocol.py in get_return_value (answer, gateway_client, target_id, name) 
 326 raise Py4JJavaError(
 327 "An error occurred while calling {0}{1}{2}.\n" . 
 --> 328 format(target_id, ".", name), value)
 329 else : 
 330 raise Py4JError(

 Py4JJavaError : An error occurred while calling o264.load.
: com.microsoft.sqlserver.jdbc.SQLServerException: Database 'edsqutarmsqluw2005' on server 'sqlqutarmsvruw2003.database.windows.net' is not currently available. Please retry the connection later. If the problem persists, contact customer support, and provide them the session tracing ID of '{BB580F67-5B9E-4DB9-A467-DB4BCB5BA171}'. ClientConnectionId:759c9207-b6c4-4cb3-b7b4-4f6bfcbaa60a
	at com.microsoft.sqlserver.jdbc.SQLServerException.makeFromDatabaseError(SQLServerException.java:258)
	at com.microsoft.sqlserver.jdbc.TDSTokenHandler.onEOF(tdsparser.java:256)
	at com.microsoft.sqlserver.jdbc.TDSParser.parse(tdsparser.java:108)
	at com.microsoft.sqlserver.jdbc.SQLServerConnection.sendLogon(SQLServerConnection.java:4290)
	at com.microsoft.sqlserver.jdbc.SQLServerConnection.logon(SQLServerConnection.java:3157)
	at com.microsoft.sqlserver.jdbc.SQLServerConnection.access$100(SQLServerConnection.java:82)
	at com.microsoft.sqlserver.jdbc.SQLServerConnection$LogonCommand.doExecute(SQLServerConnection.java:3121)
	at com.microsoft.sqlserver.jdbc.TDSCommand.execute(IOBuffer.java:7151)
	at com.microsoft.sqlserver.jdbc.SQLServerConnection.executeCommand(SQLServerConnection.java:2478)
	at com.microsoft.sqlserver.jdbc.SQLServerConnection.connectHelper(SQLServerConnection.java:2026)
	at com.microsoft.sqlserver.jdbc.SQLServerConnection.login(SQLServerConnection.java:1687)
	at com.microsoft.sqlserver.jdbc.SQLServerConnection.connectInternal(SQLServerConnection.java:1528)
	at com.microsoft.sqlserver.jdbc.SQLServerConnection.connect(SQLServerConnection.java:866)
	at com.microsoft.sqlserver.jdbc.SQLServerDriver.connect(SQLServerDriver.java:569)
	at org.apache.spark.sql.execution.datasources.jdbc.JdbcUtils$$anonfun$createConnectionFactory$1.apply(JdbcUtils.scala:64)
	at org.apache.spark.sql.exec

In [0]:
#Forcing data to be float
Raw_Total_Data.iloc[:, 1:] = Raw_Total_Data.iloc[:, 1:].apply(pd.to_numeric, downcast='float', errors='coerce').astype(float)
for data in Raw_Total_Data.columns[Raw_Total_Data.isna().any()].tolist():
  for i, row in Raw_Total_Data.iterrows():
    val = Raw_Total_Data.at[i, data]
    if math.isnan(val):
      print("Column: " + data)
      print("Index: " + str(i))
Cleaned_Total_Data = Raw_Total_Data.iloc[:,1:].dropna()

Raw_Target_Data.iloc[:, 1:] = Raw_Target_Data.iloc[:, 1:].apply(pd.to_numeric, downcast = 'float', errors = 'coerce').astype(float)
Cleaned_Target_Data = Raw_Target_Data.iloc[:,1:].dropna()

n_features = Cleaned_Total_Data.shape[1]
n_samples = Cleaned_Total_Data.shape[0]

print('Total number of features: ', str(n_features))
print('Total number of samples: ', str(n_samples))

Dates = Raw_Target_Data['Sample_Date']
Dates.columns = ['Sample_Date']
dropped_values = Dates.shape[0] - Cleaned_Target_Data.shape[0]

#Backlog item... oldest date values are currently dropped. If all values are correctly formatted (ie once integration is complete) no dates will be dropped
Dates = Dates.iloc[(dropped_values):]

# Dates.head

Column: Antimony_mg_L
Index: 27
Column: Barium_mg_L
Index: 27
Column: Beryllium_mg_L
Index: 27
Column: Cadmium_g_L
Index: 27
Column: Chromium_mg_L
Index: 27
Column: Iron_mg_L
Index: 27
Column: Lead_mg_L
Index: 27
Column: Lithium_mg_L
Index: 27
Column: Molybdenum_mg_L
Index: 27
Column: Nickel_mg_L
Index: 27
Column: Selenium_mg_L
Index: 27
Column: Silver_mg_L
Index: 27
Column: Thallium_mg_L
Index: 27
Column: Uranium_mg_L
Index: 27
Column: Vanadium_mg_L
Index: 27
Column: Zinc_mg_L
Index: 27
Column: Aluminum_Dissolved_mg_L
Index: 27
Total number of features: 82
Total number of samples: 235

In [0]:

import warnings
# Squash warning messages
warnings.showwarning = lambda *args, **kwargs: None

#This is a robust clustering algorithm used to detect outliers
Birch_Output_System = sklearn.cluster.Birch(n_clusters=2, compute_labels=True, copy=True).fit_predict(Cleaned_Total_Data)
# Birch_Output_System3 = sklearn.cluster.Birch(n_clusters=3, compute_labels=True, copy=True).fit_predict(Cleaned_Total_Data)

#Applying to one data stream at a time
Birch_Output_Individual_Stream = Cleaned_Total_Data.copy()
# Birch_Output_Individual_Stream3 = Cleaned_Total_Data.copy()

for Counter in range(0, Cleaned_Total_Data.shape[1]):
    Input_Array = Cleaned_Total_Data.iloc[:,Counter].values
    Birch_Output_Individual_Stream.iloc[:, Counter] = sklearn.cluster.Birch(n_clusters=2, compute_labels=True, copy=True).fit_predict(Input_Array.reshape(-1,1))
#     Birch_Output_Individual_Stream3.iloc[:, Counter] = sklearn.cluster.Birch(n_clusters=3, compute_labels=True, copy=True).fit_predict(Input_Array.reshape(-1,1))

In [0]:
#Third algorithm is agglomorative clustering
#Applying to the whole system
AggCluster_Output_System = sklearn.cluster.AgglomerativeClustering(n_clusters=2, affinity='euclidean', compute_full_tree='auto', linkage='ward').fit_predict(Cleaned_Total_Data)

#Applying to one data stream at a time
AggCluster_Output_Individual_Stream = Cleaned_Total_Data.copy()
for Counter in range(0, Cleaned_Total_Data.shape[1]):
    Input_Array = Cleaned_Total_Data.iloc[:,Counter].values
    AggCluster_Output_Individual_Stream.iloc[:, Counter] = sklearn.cluster.AgglomerativeClustering(n_clusters=2, affinity='euclidean', compute_full_tree='auto', linkage='ward').fit_predict(Input_Array.reshape(-1,1))

In [0]:
#Applying the ensembling algoritm

#Analyzing overall system state
#First agglomerating features
Ensemble_System_Input = np.zeros((int(AggCluster_Output_System.shape[0]),3))
Ensemble_System_Input[:,0] = AggCluster_Output_System
Ensemble_System_Input[:,1] = Birch_Output_System
Ensemble_System_TempStorage = sklearn.cluster.FeatureAgglomeration(n_clusters=2, affinity='euclidean', 
                                                              compute_full_tree='auto', linkage='ward').fit_transform(Ensemble_System_Input)
#Then applying clustering
Ensemble_Output_System = sklearn.cluster.AgglomerativeClustering(n_clusters=2, affinity='euclidean', compute_full_tree='auto', linkage='ward').fit_predict(Ensemble_System_TempStorage)

#Analyzing Individual Data Streams
Ensemble_Output_Individual_Stream = Cleaned_Total_Data.copy()
for Counter in range(0, Cleaned_Total_Data.shape[1]):
    Ensemble_Input_Individual_Stream = np.zeros((int(AggCluster_Output_System.shape[0]),3))
    Ensemble_Input_Individual_Stream[:,0] = AggCluster_Output_Individual_Stream.iloc[:,Counter].values
    Ensemble_Input_Individual_Stream[:,1] = Birch_Output_Individual_Stream.iloc[:,Counter].values
    Ensemble_Individual_Stream_TempStorage = sklearn.cluster.FeatureAgglomeration(n_clusters=2, affinity='euclidean', 
                                                                       compute_full_tree='auto', linkage='ward').fit_transform(Ensemble_Input_Individual_Stream)
    Ensemble_Output_Individual_Stream.iloc[:, Counter] = sklearn.cluster.AgglomerativeClustering(n_clusters=2, affinity='euclidean', compute_full_tree='auto', linkage='ward').fit_predict(Ensemble_Individual_Stream_TempStorage)


    

In [0]:
#Metrics calculations for debugging and/or experimenting

# EnsembleModel_Silhoutte = metrics.silhouette_score(Cleaned_Total_Data, Ensemble_Output_System, metric='euclidean')
# Aggcluster_Silhoutte = metrics.silhouette_score(Cleaned_Total_Data, AggCluster_Output_System, metric='euclidean')
# Birch_Silhoutte = metrics.silhouette_score(Cleaned_Total_Data, Birch_Output_System, metric='euclidean')
# EnsembleModel_db = metrics.davies_bouldin_score(Cleaned_Total_Data, Ensemble_Output_System)
# Aggcluster_db = metrics.davies_bouldin_score(Cleaned_Total_Data, AggCluster_Output_System)
# Birch_db = metrics.davies_bouldin_score(Cleaned_Total_Data, Birch_Output_System)

# print(Aggcluster_db)

In [0]:
#Saving the results of Anomaly Detection Algorithm
#These options are left here in case a different approach is decided upon later
# Ensemble_Output_System_Dataframe = pd.DataFrame(data= Ensemble_Output_System, index = Cleaned_Total_Data.index.values)
# Birch_Output_System_Dataframe = pd.DataFrame(data= Birch_Output_System, index = Cleaned_Total_Data.index.values)
# Ensemble_Output_Individual_Stream_Dataframe = pd.DataFrame(data= Ensemble_Output_Individual_Stream, index = Cleaned_Total_Data.index.values,
#                                     columns = Cleaned_Total_Data.columns.values)


# Birch_Output_Individual_Stream_Dataframe = pd.DataFrame(data= Birch_Output_Individual_Stream, index = Cleaned_Total_Data.index.values,
#                                     columns = Cleaned_Total_Data.columns.values)


Agg_Output_System_Dataframe = pd.DataFrame(data= AggCluster_Output_System, index = Cleaned_Total_Data.index.values)

Agg_Output_Individual_Stream_Dataframe = pd.DataFrame(data= AggCluster_Output_Individual_Stream, index = Cleaned_Total_Data.index.values,
                                    columns = Cleaned_Total_Data.columns.values)


#Title overall system cluster output as system status
Agg_Output_System_Dataframe.columns = ['System_Status']

#Append the dates and two output clusters horizontally (axis 1 not 0)
Agg_Output_All = pd.concat([Dates, Agg_Output_System_Dataframe, Agg_Output_Individual_Stream_Dataframe], axis=1)
Agg_Output_All = Agg_Output_All.dropna()

# Agg_Output_All.head()


In [0]:
# Go through each column, invert value if the clustering is inverted
# rows is the total number of tests in the set
# the values in each column are 1 or 0
# if the sum of the values in each column are less than 1/2(rows)
# flip the 1's and 0's
half_point = 0.5 * len(Agg_Output_All)

# Run on every column except the date column
for col in Agg_Output_All.columns[1:]:
  if Agg_Output_All[col].sum() < half_point:
    for i, row in Agg_Output_All.iterrows():
      val = Agg_Output_All.at[i, col]
      if val == 0:
        Agg_Output_All.at[i, col] = 1
      elif val == 1:
        Agg_Output_All.at[i, col] = 0
        

In [0]:
#convert final results back to Spark dataframe
Agg_Output_SDF = spark.createDataFrame(Agg_Output_All)

# Writing results to IWW Tag Value table for powerBI ingestion
Agg_Output_SDF.write.format("jdbc").mode("overwrite") \
    .option("driver", "com.microsoft.sqlserver.jdbc.SQLServerDriver")\
    .option("url", "jdbc:sqlserver://sqlqutarmsvruw2003.database.windows.net;databaseName=edsqutarmsqluw2005") \
    .option("dbtable", "ANOMALY_CLUSTERS") \
    .option("user", "ABO_IWWD_Admin") \
    .option("password", "!WWD_ad_2019").save()


In [0]:
# Iterate through each row of the dataframe
# Calculate TP, TN, FP, FN, Sensitivity, Specificity
# Sensitivity = TP / TP + FN
# Specificity = TN / FP + TN
# Add calulcated values to a new table with format:
#
# Test Type        TP      TN      FP        FN      Sens         Spec
# Total Alkalinity
# Conductivity
# .......
rows = len(Agg_Output_All)
tests = Agg_Output_All.columns[2:]
blanks = [0.0] * len(tests)

data = {'Tests': tests,
       'True Positive': blanks,
       'True Positive %': blanks,
       'True Negative': blanks,
       'True Negative %': blanks,
       'False Positive': blanks,
       'False Positive %': blanks,
       'False Negative': blanks,
       'False Negative %': blanks,
       'Sensitivity': blanks,
       'Sensitivity Rank': blanks,
       'Specificity': blanks,
       'Specificity Rank': blanks}
alg_accuracy = pd.DataFrame(data)

ctr = 0
for col in Agg_Output_All.columns[2:]:
  tp = 0
  tn = 0
  fp = 0
  fn = 0
  # sum up the tp, tn, fp, and fn for each row
  for i, row in Agg_Output_All.iterrows():
    sys_status = Agg_Output_All.at[i, 'System_Status']
    el_status = Agg_Output_All.at[i, col]
    if sys_status == 1 and el_status == 1:
      tp += 1
    elif sys_status == 0 and el_status == 0:
      tn += 1
    elif sys_status == 0 and el_status == 1:
      fp += 1
    elif sys_status == 1 and el_status == 0:
      fn += 1
  
  # assign values to the alg_accuracy dataframe
  alg_accuracy.at[ctr, 'True Positive'] = tp
  alg_accuracy.at[ctr, 'True Positive %'] = float(tp / rows) * 100.0
  alg_accuracy.at[ctr, 'True Negative'] = tn
  alg_accuracy.at[ctr, 'True Negative %'] = float(tn / rows) * 100.0
  alg_accuracy.at[ctr, 'False Positive'] = fp
  alg_accuracy.at[ctr, 'False Positive %'] = float(fp / rows) * 100.0
  alg_accuracy.at[ctr, 'False Negative'] = fn
  alg_accuracy.at[ctr, 'False Negative %'] = float(fn / rows) * 100.0

  alg_accuracy.at[ctr, 'Sensitivity'] = float(tp) / float(tp + fn) * 100.0
  alg_accuracy.at[ctr, 'Specificity'] = float(tn) / float(fp + tn) * 100.0
  
  ctr += 1

# Get the ranking of sensitivity and specificity for each element
sens_rank = len(alg_accuracy['Sensitivity']) - stats.rankdata(alg_accuracy['Sensitivity'], method='max')
spec_rank = len(alg_accuracy['Specificity']) - stats.rankdata(alg_accuracy['Specificity'], method='max')
for i, row in alg_accuracy.iterrows():
  alg_accuracy.at[i, 'Sensitivity Rank'] = sens_rank[i]
  alg_accuracy.at[i, 'Specificity Rank'] = spec_rank[i]

In [0]:
cols = alg_accuracy.columns[1:]
alg_accuracy[cols] = alg_accuracy[cols].apply(pd.to_numeric, downcast = 'float', errors = 'coerce')

#convert final results back to Spark dataframe
alg_accuracy_SDF = spark.createDataFrame(alg_accuracy)

# Writing results to IWW Tag Value table for powerBI ingestion
alg_accuracy_SDF.write.format("jdbc").mode("overwrite") \
    .option("driver", "com.microsoft.sqlserver.jdbc.SQLServerDriver")\
    .option("url", "jdbc:sqlserver://sqlqutarmsvruw2003.database.windows.net;databaseName=edsqutarmsqluw2005") \
    .option("dbtable", "ALGORITHM_ACCURACY") \
    .option("user", "ABO_IWWD_Admin") \
    .option("password", "!WWD_ad_2019").save()

In [0]:
# Create an Attribute Table
# Get the types of elemental tests that are run, remove the date column
cluster_type = list(Agg_Output_All)
cluster_type.remove('Sample_Date')

# Write the test types to a dataframe
clustering_type_dataframe = pd.DataFrame(data=cluster_type, columns=['Clustering Category'])
# clustering_type_dataframe.head()

In [0]:
# Write the Test Type dataframe to a new table in the database
iww_clustering_types_SDF = spark.createDataFrame(clustering_type_dataframe)
iww_clustering_types_SDF.write.format("jdbc").mode("overwrite") \
    .option("driver", "com.microsoft.sqlserver.jdbc.SQLServerDriver")\
    .option("url", "jdbc:sqlserver://sqlqutarmsvruw2003.database.windows.net;databaseName=edsqutarmsqluw2005") \
    .option("dbtable", "CLUSTERING_TYPE") \
    .option("user", "ABO_IWWD_Admin") \
    .option("password", "!WWD_ad_2019").save()

In [0]:
# Create a Bridge Table
bridge = pd.melt(Agg_Output_All,
                    id_vars=['Sample_Date'],
                    value_vars=cluster_type,
                    var_name='Attribute',
                    value_name='Value')

bridge['Value'] = bridge['Value'].apply(pd.to_numeric, downcast='float', errors='coerce').astype(float)

In [0]:
# Write the Bridge dataframe to a new table in the database
iww_bridge_SDF = spark.createDataFrame(bridge)
iww_bridge_SDF.write.format("jdbc").mode("overwrite") \
    .option("driver", "com.microsoft.sqlserver.jdbc.SQLServerDriver")\
    .option("url", "jdbc:sqlserver://sqlqutarmsvruw2003.database.windows.net;databaseName=edsqutarmsqluw2005") \
    .option("dbtable", "CLUSTERING_TYPE_BRIDGE") \
    .option("user", "ABO_IWWD_Admin") \
    .option("password", "!WWD_ad_2019").save()

In [0]:
# #Reading Toxicity data table
# toxicity_data_SDF = spark.read.format("jdbc") \
# 	.option("driver", "com.microsoft.sqlserver.jdbc.SQLServerDriver")\
# 	.option("url", "jdbc:sqlserver://sqlqutarmsvruw2003.database.windows.net;databaseName=edsqutarmsqluw2005") \
# 	.option("dbtable", "dbo.TOXICITY_DATA$") \
# 	.option("user", "ABO_IWWD_Admin") \
# 	.option("password","!WWD_ad_2019").load()

# #Using Pandas dataframes with arrow conversion
# toxicity_data = toxicity_data_SDF.select("*").toPandas()

In [0]:
# print(toxicity_data['Sample Date'])
# cols = toxicity_data.columns[3:]
# toxicity_data[cols] = toxicity_data[cols].apply(pd.to_numeric, downcast = 'float', errors = 'coerce')
# print(toxicity_data['Sample Date'])

# #convert final results back to Spark dataframe
# toxicity_data_SDF = spark.createDataFrame(toxicity_data)

# # Writing results to IWW Tag Value table for powerBI ingestion
# toxicity_data_SDF.write.format("jdbc").mode("overwrite") \
#     .option("driver", "com.microsoft.sqlserver.jdbc.SQLServerDriver")\
#     .option("url", "jdbc:sqlserver://sqlqutarmsvruw2003.database.windows.net;databaseName=edsqutarmsqluw2005") \
#     .option("dbtable", "TOXICITY_DATA") \
#     .option("user", "ABO_IWWD_Admin") \
#     .option("password", "!WWD_ad_2019").save()